In [1]:
import numpy as np
import math
import sys
from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from PIL import Image,ImageDraw
import cv2

In [2]:
def build_dict(detection_name,fN,angles):
    d = {}
    d[fN] = {}

    with open(detection_name,'r') as file:
        for i,line in enumerate(file):
            line = line.strip()

            if 'tiff' in line:

                fN = line[:-5]
#                 print('fN',fN)
            elif 'angle' in line:
                angle = line.split()[1]
                angles.add(int(angle))
#                 print('angle',angle)
            else:
                if len(line.split())<2:
#                     print('only one',line.split(),len(line.split()),i)
                    n = int(line)
                    continue
                else:
#                     print(d)
                    if angle not in d[fN].keys():
                        d[fN][angle] = []
                    d[fN][angle].append(line.split())
    return d,angles

            
# d

In [3]:
def coord_words(fN,ids):
    #directory with bounding boxes(new ones cos we're only checking for valid)
    coords_old = np.load('../../torch-phoc/mudit/training_new/'+fN+'.npy')
    #directory with npy file with before,after,ground truth words
    words_all = np.load('../../torch-phoc/mudit/training_mudit/'+fN+'.npy')
    words_old = words_all.T[1]
    gt = words_all.T[2]
#     print(zip(words_old,gt))
    return coords_old[ids],words_old,gt

In [4]:
def find_closest_angle(angle,angles):
    m = sys.maxint
    closest = 0
    angle = abs(angle)
    for a in angles:
        if abs(a-angle)<m:
            m = abs(a-angle)
            closest = a
    return closest

In [5]:
def find_slope(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    if x2-x1 == 0:
#         print('0',p1,p2)
        return 90
    angle = np.rad2deg(np.arctan((y2-y1)/(x2-x1)))
    return angle

In [6]:
def compute_angle(coords):
    bl,br,tr,tl = coords
    d1 = find_dist(bl,br)
#     print('d1',d1)
    d2 = find_dist(br,tr)
#     print('d2',d2)
    d3 = find_dist(tr,tl)
    d4 = find_dist(tl,bl)
#     print('d3,d4',d3,d4)
    if(d1!=d3 or d2!=d4):
#         print('not equal',d1,d2,d3,d4)
        pass
        #not too different
    if d1>d2:
        p1 = bl
        p2 = br
    else:
        p1 = br
        p2 = tr
    slope = find_slope(p1,p2)
    return slope

In [7]:
def find_dist(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return math.hypot(x2-x1,y2-y1)

In [8]:
def find_points_start(point,ind,string,angle,d1,d2):
    m = np.tan(np.deg2rad(angle))
    x,y = point
#     print('dist',(float(ind)/len(string))*d1,string,ind,d1,d2)
    blx = x+((float(ind)/len(string))*d1)*(np.sqrt(1/(1+m**2)))
    bly = y+((float(ind)/len(string))*d1)*m*(np.sqrt(1/(1+m**2)))
    brx = x+((float(ind+1)/len(string))*d1)*(np.sqrt(1/(1+m**2)))
    bry = y+((float(ind+1)/len(string))*d1)*m*(np.sqrt(1/(1+m**2)))
    m2 = np.tan(np.deg2rad(angle+90))
    trx = brx-d2*(np.sqrt(1/(1+m2**2)))
    try_ = bry-d2*m2*(np.sqrt(1/(1+m2**2)))     
    tlx = blx-d2*(np.sqrt(1/(1+m2**2)))
    tly = bly-d2*m2*(np.sqrt(1/(1+m2**2)))
    return [(blx,bly),(brx,bry),(trx,try_),(tlx,tly)]

In [9]:
def find_char(coords,string,c,d,angle):
    count = c[char]
    new_coords = [(c1,c2) for [c1,c2] in coords]
#     new_coords = [(0,0),(2,0),(2,2),(0,2)]
    poly = Polygon(new_coords)
    vals = []
    new_points = None
    for angle in angles:
        points = d[fN][str(angle)]
        for p in points:
            blx = float(p[0])
            bly = float(p[1])
            w = float(p[2])
            h = float(p[3])
            m = np.tan(np.deg2rad(angle))
            brx = blx+w*(np.sqrt(1/(1+m**2)))
            bry = bly+w*m*(np.sqrt(1/(1+m**2)))
            m2 = np.tan(np.deg2rad(angle+90))
            trx = brx+h*(np.sqrt(1/(1+m2**2)))
            try_ = bry+h*m2*(np.sqrt(1/(1+m2**2)))     
            tlx = blx+h*(np.sqrt(1/(1+m2**2)))
            tly = bly+h*m2*(np.sqrt(1/(1+m2**2)))
            all_ps = [(blx,bly),(brx,bry),(trx,try_),(tlx,tly)]
#             if(not(poly.contains(Point[all_ps[0]]) or poly.contains(Point[all_ps[1]]) or poly.contains(Point[all_ps[2]]) or poly.contains(Point[all_ps[3]]))):
                
                    
            for all_p in all_ps:
                point = Point(all_p)
#     #     point = Point(0,1)
                if(poly.contains(point)):
                    count-=1
#                     print('found',count)
                    vals.append(all_p)
    if len(vals)>0:
        if count<=0:
            return []
    else:
            vals = sorted(vals)
            start = 0
            for i in range(len(vals)):
                ind = string[start:].index(char)
                start = ind
            while(count>0):
                ind = string[start:].index(char)
                start = ind
                bl,br,tr,tl = coords
                d1 = find_dist(bl,br)
            #     print('d1',d1)
                d2 = find_dist(br,tr)
                count-=1
                if d1>d2:
                    new_points = find_points_start(bl,ind,string,find_slope(bl,br),d1,d2)
#                     print(new_points)
                else:
                    new_points = find_points_start(br,ind,string,find_slope(br,tr),d2,d1)
#                     print(new_points)
    return new_points
        
#                     print(poly.contains(point),new_coords,all_p,angle)
#     print(count,coords,angle)

In [10]:
def return_points(fN,ids):
    coords_old,words_old,gt = coord_words(fN,ids)
    cor = 0
    wr = 0
    all_points = []
    for i,coords in enumerate(coords_old):
#         print(i)
        a = compute_angle(coords)
        angle = find_closest_angle(a,angles)
        word = words_old[i].lower()
        gt_i = gt[i].lower()
        old_coords = coords
        if word==gt_i[1:] or word==gt_i[:-1] or word==gt_i:
            c = Counter(word)
#             if gt_i == 'osage':
#                 print('osagooooooooo',coords)
            if c.has_key(char):
    #             print word,gt_i,angle
#                 print('s',gt_i)
                new_points = find_char(old_coords,gt_i,c,d,angle)
                if(new_points):
#                     if gt_i=='osage':
#                         print('osagooooo!',old_coords)
                    all_points.append((new_points,gt_i,old_coords))
            cor+=1
        else:
            wr+=1
    return all_points
# print(cor,wr)

In [11]:
def save_map(fN,all_points,char):
    map_name='../'+fN+'_s.tiff'
    im = Image.open(map_name)
    img = np.array(im)
#     print(all_points)
    for points,string,coords in all_points:
    #             print(coords)
    #             print(coord[0],'0',coord[1])
            points_all = [[int(p[0]),int(p[1])] for p in points]
            points_all = np.array(points_all,dtype='int32')
    #         points_all = points_all.reshape((-1,1,2))
#             print(points_all,string,coords)
    #         if string=='osage':
    #             print('neoshoooooo',coords)
    #             cv2.polylines(img,np.int32([points]),True,(0,0,255),10)
    #             cv2.polylines(img,np.int32([coords]),True,(255,0,0),10)
    #         else:
    #             pass
            cv2.polylines(img,np.int32([points]),True,(0,255,0),10)
    #         cv2.polylines(img,np.int32([coords]),True,(255,0,0),10)
    #     If each character has to be saved:
    im = Image.fromarray(img)
    im.save(char+fN+'.tiff')

In [12]:
#give all characters
all_chars = ['s','e']
#give all file names/ crawl directory for file names of mas
fns = ['D0090-5242001','D0090-5242002']
for fN in fns:
    for char in all_chars:
        angles = set()
        #the word-det files with output from char frcnn
        d,angles = build_dict('words-det-'+fN+'-01-'+char+'.txt',fN,angles)
        #images to extend info with ids, comment not used for now
        extend = np.load('../../torch-phoc/mudit/training_images/image_dir_'+fN+'.npy')
        # ids = extend[0]
        #we just use all the valid new images
        ids = [i for i in range(469)] 
        all_points = return_points(fN,ids)
        #saves in same directory, our final output in npy, change name if required
        np.save(char+fN+'allpoints.npy',np.array(all_points))
        save_map(fN,all_points,char)

In [13]:
pass
# im = Image.open('../'+fN+'_s.tiff')
# imArray = np.asarray(im)
# for p in all_points[:1]:
    
#     maskIm = Image.new('L',(imArray.shape[1],imArray.shape[0]),0)
#     ImageDraw.Draw(maskIm).polygon(p,outline=1,fill=1)
#     mask = np.array(maskIm)
#     newIm = np.empty(imArray.shape,dtype='uint8')
#     newIm[:,:,:3] = imArray[:,:,:3]
#     newIm[:,:,3] = mask*255
#     newImg = Image.fromarray(newIm,'RGBA')
#     newImg.save('first.tiff')

In [14]:
print(angles,d.keys())

(set([0, 70, 40, 10, 80, 50, 20, 90, 60, 30]), ['D0090-5242002'])
